In [1]:
from preproc import PreProcessor
from ensemble_model import EnsembleModel
from bert_model import Model

pp = PreProcessor()

# Downsample
pp.downsample_neg()

[nltk_data] Downloading package stopwords to
[nltk_data]     /homes/ss5120/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [2]:
ensemble = EnsembleModel(pp.raw_dev)

In [3]:
m1 = Model("baseline", layers_to_freeze=4, num_train_epochs=2)
m2 = Model("baseline", layers_to_freeze=6, num_train_epochs=2)
m3 = Model("baseline", layers_to_freeze=6, num_train_epochs=2)
m4 = Model("baseline", layers_to_freeze=6, num_train_epochs=2)
m5 = Model("baseline", layers_to_freeze=5, num_train_epochs=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN th

In [4]:
import pandas as pd

d1 = pd.read_csv("aug_datasets/bert_uncased.csv")
pp.augmented = d1
d1 = pp.run_preprocess(
    white_func=pp.whitespace_norm,
    punc_func=pp.remove_punc,
    dig_func=pp.remove_digits,
)
d2 = pd.read_csv("aug_datasets/bt.csv")
pp.augmented = d2
d2 = pp.run_preprocess(
    white_func=pp.whitespace_norm,
    punc_func=pp.remove_punc,
    dig_func=pp.remove_digits,
)
d3 = pd.read_csv("aug_datasets/bt_crop.csv")
pp.augmented = d3
d3 = pp.run_preprocess(
    white_func=pp.whitespace_norm,
    punc_func=pp.remove_punc,
    dig_func=pp.remove_digits,
)
d4 = pd.read_csv("aug_datasets/gpt2.csv")
pp.augmented = d4
d4 = pp.run_preprocess(
    white_func=pp.whitespace_norm,
    punc_func=pp.remove_punc,
    dig_func=pp.remove_digits,
)
d5 = pd.read_csv("aug_datasets/gpt2_crop_bt.csv")
pp.augmented = d5
d5 = pp.run_preprocess(
    white_func=pp.whitespace_norm,
    punc_func=pp.remove_punc,
    dig_func=pp.remove_digits,
)

In [5]:
ensemble.add_model(m1, d1)
ensemble.add_model(m2, d2)
ensemble.add_model(m3, d3)
ensemble.add_model(m4, d4)
ensemble.add_model(m5, d5)

In [6]:
ensemble.train()

Map:   0%|          | 0/11551 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.318900,0.667525,0.855778,0.452899,0.354108,0.628141
2,0.218000,0.225103,0.923591,0.518072,0.646617,0.432161


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Map:   0%|          | 0/11551 [00:00<?, ? examples/s]

...

Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Map:   0%|          | 0/11551 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.354600,0.233779,0.909742,0.422018,0.539062,0.346734
2,0.237900,0.235842,0.921681,0.464052,0.663551,0.356784
3,0.099100,0.388188,0.917383,0.483582,0.595588,0.407035


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [7]:
majority_voted = ensemble.majority_vote(pp.raw_dev[['text']])

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [8]:
ensemble.compute_metrics(majority_voted, pp.raw_dev[['label']])

{'accuracy': 0.9283667621776505,
 'f1': 0.5454545454545454,
 'precision': 0.6870229007633588,
 'recall': 0.45226130653266333}